**Intro to Preprocessing and Feature Extraction** | CortexCodec

EEG Motor Movement/Imagery Dataset (Sept. 9, 2009, midnight)

A set of 64-channel EEGs from subjects who performed a series of motor/imagery tasks has been contributed to PhysioNet by the developers of the BCI2000 instrumentation system for brain-computer interface research.

When using this resource, please cite the original publication:
Schalk, G., McFarland, D.J., Hinterberger, T., Birbaumer, N., Wolpaw, J.R. BCI2000: A General-Purpose Brain-Computer Interface (BCI) System. IEEE Transactions on Biomedical Engineering 51(6):1034-1043, 2004.


Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C., Mark, R., ... & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals. Circulation [Online]. 101 (23), pp. e215–e220. RRID:SCR_007345.

In [ ]:
# install python packages using pip (standard package installer for Python)
# you are allowed to add more if you think it will be useful for your process

!pip -q install matplotlib
!pip -q install scipy
!pip -q install mne
!pip -q install numpy

In [ ]:
# import required packages (fill in the blanks!)
# you may not use all of these
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
import pywt
import ??
import ?? as np


The most important part of working with data is making sure you have data to work with. To load .edf files (or pretty much any EEG file) first download the data and then import the file to your Drive. Then, we want to allow the Colab to access our Drive so that we can choose the specific file to work with.

I recommend making a specific folder in your drive for CC files so that it is easier to navigate. After mounting your data, you should see a file path, specifically linked to your drive, appear on the left under the files tab.


https://physionet.org/content/eegmmidb/1.0.0/S001/#files-panel
This is the dataset we'll be using! Please navigate through this, download S001R01.edf, and import that file to your Google Drive. Then, copy the file path from your files tab on Colab and paste that where it says 'file_path = ???'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from the files tab, copy your specific file path
file_path = ???

https://mne.tools/stable/auto_tutorials/intro/10_overview.html#tut-overview

Here's the cheatsheet for a lot of the work we'll be doing. **Please** try to do this work on your own! But if you find yourself struggling you can reference this page.

In [ ]:
# lets get a visualization of our raw data! reference https://mne.tools/stable/api/reading_raw_data.html to fill in the blanks
# we set preload to true to save the data to our memory. you may also use raw.load_data() instead but both are essentially the same for our purposes
raw = mne.io.???(???, preload=True)

# raw.plot will be how we plot our raw data, its useful for understanding if the file we're using is suitable for anaylsis
# please plot the first 15-25 channels of our data (but you can play around with your n_channels value)

raw.plot(n_channels = ??, scalings='auto', title='Raw EEG Data Visualization')

In [ ]:
# lets learn a little bit about the data we're working with

info = raw.info
print(info)

In [ ]:
fs = ??? # you can also find this with raw.info['sfreq']!
#but for the purposes of this assignment reference your data from raw.info to find what the sampling freq. of our data is

# how many channels are in our data? ______
# what is the original highpass and lowpass frequencies? _____ and _____

Now that we know a little more about the data we're working with, lets start cleaning it. For our purposes, we will be trying to use ICA and PCA (which I will teach you guys how to do manually in another session as its a little more complicated than ICA).

For now, lets look at bandpass filtering. A bandpass filter allows a specific range of frequencies to pass through, removing both low frequency and high frequency articfacts. Its usually pretty effective to have your first step of preprocessing include a bandpass filter, just because its a simple way to remove prominent artifacts.

https://mne.tools/stable/generated/mne.io.Raw.html

In [ ]:
# apply a bandpass filter from what you think would be the best for our data based on your research!
# feel free to play around with this but make sure you reset the raw data each time so you're not filtering already filtered data
raw.???(low_freq=?, high_freq=?)

In [ ]:
# please do 5 channels this time :)
raw.plot(n_channels = ???, scalings='auto', title='Filtered EEG Data Visualization')

🤯 Our visualization looks a little crazier than our raw data. But, if we're cleaning our signal, why does it seem to be "worse"? This is due to the fact that a bandpass filter can make the amplitude of our signals *appear* to change by removing competing frequencies. When you apply a bandpass filter, you can remove frequency components that may be canceling out or obscuring our true signal. That's why our resulting data has a higher peak amplitude: because the targeted frequency is no longer being masked.

We also zoomed in on our data, only looking at 5 channels now.

In [ ]:
# now lets move on to cleaning an individual channel. its definitely possible to clean more than one channel at once, but for the purpose of this assignment:
# determine what the first channel in this dataset is called: _____

print(raw.ch_names)

In [ ]:
# this is an easier way of figuring out what you did in the last cell! if its our first channel, what shoud go in our index slot?
channel_name = raw.ch_names[??]
print(channel_name)

We'll move on to our application of ICA now. This will be our primary form of preprocessing data! It is the most common preprocessing tenchique used in functional neural data anaylsis.

ICA splits the EEG signals into independent components using information theory methods (statistical analysis of subcomponents). We will talk about this during our meeting!

In [ ]:
# time for ICA! https://mne.tools/stable/generated/mne.preprocessing.ICA.html
from mne.preprocessing import ICA

# i provided some values for our model, but read the MNE ICA link to figure out what each of these parameters mean in terms of our data
# what are the purposes of 'n_components', 'random_state', and 'max_iter' in your own words? write one sentence for each:



ica = mne.??(n_components=15, random_state=97, max_iter=800)
# lets fit our ICA model to our raw data
ica.???(raw)

In [ ]:
# now that we've fitted the data to our analysis model, lets apply it!

raw_clean = ???(raw.copy())

In [ ]:
# please extract data for plotting purposes (reference the mne.raw link if you're stuck)
# we'll be comparing the first channel of our dataset, so please put the first array index in the []

data_raw = raw.???()[???]
data_clean = raw_clean.???()[??]

In [ ]:
time = np.arange(len(data_raw)) / fs # this helps creates an array of timestamps corresponding to a sampled signal
# we must generate a time axis to plot signal processing
# if you have questions on how this line of code works, talk to me at office hours this week! but for now don't worry too much about it, it'll look the same for most of our work

In [ ]:
print("Analyzing channel: " + str(???))
print("Sampling rate: " + str(???) + "Hz")

In [ ]:
# reset raw to its rawest form, as we previously applied ICA and a bandpass filter
raw = mne.io.read_raw_edf(file_path, preload=True)

In [ ]:
# lets plot our cleaned data against our raw data so we can see what all our work has done so far
plt.figure(figsize=(15, 5))
# scale up data, as MNE plots in microvolts instead of volts (which is the form our data is in) --> what power of 10 do we multiply our data by?
# notice that this is in the form of (x, y, label, color) this should help you later
plt.plot(time, data_raw * ???, label='Raw', color='red')
plt.plot(time, data_clean * ???, label='Cleaned', color='blue')



# these should be your standard plotting procedures! always include a title, axis labels, and a legend at the very least. you can copy this later on
plt.???('EEG Comparison – Channel: ' + str(???))
plt.???('???') # x axis
plt.???('???') # y axis (include units!)
plt.???() # this should be the legend


plt.show()

Congratulations!! I hope that this demonstration of bandpass and ICA gives you a visual demonstration of why these procedures are so crucial when working with EEG data.

Lets move on to some visualizations of feature extraction with our data. Now that we have clean data, we can extract certain features to train our model with. I'm sure that you guys are more than familiar with why we do FE so I'll save you the additional lecture. Our two main forms that we'll be focusing on are discrete wavelet transform (DWT) and short-time fourier transform (STFT). We will give a demonstration in class about why we choose to use these two forms to analyze data.

This section will be dedicated to visualizing feature extraction. As we approach model training, I can teach you all how to properly use and label the features we find, but for now, lets stick to visualization to reduce the complexity of this assignment.

In [ ]:
wavelet = 'db4'
# db4 is the most popular and well-received wavelet for wavelet transform, and it designs the shape of the wavelet used to transform the data
# it stands for the Daubechies wavelets, and the “4” means it uses 4 coefficients in its filter
# feel free to learn more about why this wavelet is the standard, but its not super important for you to know :)


decomp_level = ?? # do you remember when we talked about decomposition levels? this is the max decomposition level used in our data.
# feel free to reference our past lit reviews if you forgot or want more information on what a DL is!
# set this to a respectable Dlevel based on our sampling frequency! (Hint: D1-D2 might be the most suited for our data)

coeffs = pywt.wavedec(???, ???, ???)
# this line is where the actual DWT application happens
# coeffs represents a list of approximation + detail coefficients, which we learned about when we talked about DWT. i believe i also linked a video that talks about DWT coeffs if you're still curious
# for all levels based on our *clean* data, our wavelet form, and our DL (in that order)! fill in the blanks


SyntaxError: invalid syntax (ipython-input-1692673256.py, line 7)

In [ ]:
# we need to combine our list of arrays into a matrix for analysis now
# first we need to determine how long our matrix should be first, so lets cycle through coeffs and determine the maximum length
arr_len = max(??? for c in coeffs)
# this line then pads all of the shorter arrays with 0's so that everything is the same length as the longest one
pad_coeffs = [np.pad(c, (???, arr_len - len(c))) for c in coeffs]
coeff_matrix = ???

In [ ]:
plt.imshow(np.abs(coeff_matrix), aspect='auto',
# this is essentially what we did earlier. we are determining the scaling of our axes for plotting. since DWT is plotted on a heat/energy map (scalogram) and NOT a line graph,
#we need to use plt.imshow which displays our coeffs matrix as an image
           #if you have questions as to how I found these values please come to OH to ask me or Google!
           extent=[0, len(data_clean)/fs, len(coeffs), 0])

plt.colorbar(label='|Coefficient Magnitude|') # this is the legend we use with DWT scalograms
plt.title("??? - " + str(channel_name))
plt.???("???")
plt.ylabel("Decomposition Level") #i'll give you guys this one bc it might not be as intuitive


plt.show()


Congratulations! You plotted a DWT scalogram based on data you cleaned! So proud of yall.

Take a look at the graph and see if you can visually identify any meaningful features. Write down at least 2 time stamps.

Lets move on to STFT now. Both FE methods are good, and it honestly depends on the data you're using! We will talk about when/where to use DWT vs. STFT.

In [ ]:
# this line of code is deceptively simple but i will provide some information on how it works
f, t, psd = spectrogram(???, ???, nperseg=???, noverlap=???)

# set this with a 50% overlap

The spectrogram function from scipy returns 3 outputs, frequency (f), time (t), and power spectral density (psd). It computes how the signal’s frequency content changes over time by splitting the signal into short windows and taking the FFT of each window.

Refresh yourself on what STFT and FFT are and how they work!

It'll take 4 parameters from us, data, sample freq, number of samples per window (segment length), and number of samples that overlap between windows
nperseg determines the time vs. frequency resolution:

a large nperseg → better frequency resolution, worse time resolution
a small nperseg → better time resolution, worse frequency resolution

Its all a big tradeoff, but 128-512 is usually the norm for 100-200 Hz sampling frequencies. Make sure that your nperseg is a POWER OF 2 as we are working with FFT.

Calculate overlap with noverlap/nperseg = overlap! This just ensures that we don't lose much data between windows.

In [ ]:
plt.pcolormesh(???, ???, 10 * np.log10(psd + 1e-10), shading='gouraud', cmap='jet')
# this is just plotting our spectrogram
# a spectrogram is the visual representation of STFT, which analyzes how a signal's frequency content changes over time
# FREQUENCY OVER TIME figure out which is x axis and which is y and fill in the blanks above
# the 3rd paramater just scales our psd data appropriately, and the other two are just design parameters that i filled in for you


plt.???(???) #title
plt.???(???) #x-axis label
plt.???(???) #y-axis label
plt.ylim(0, 5)  #yscale
plt.colorbar(label=???) #legend: figure how what the color densities on a spectrogram represent! hint: units are in db-HZ

plt.show()


This concludes our first homework assignment. I hope that it went well and you learned more about standard computational analysis and in respect to EEG data! I know that this data wasn't emotion-specific, but a lot of the work that we'll be doing with emotion data is incredibly similar to this baseline approach. Feel free to download more data from OpenNeuro and experiment with the libraries we used today.

I'm excited that we'll be moving on to a more technical approach, but remember all of the lit review we've done this far. As you can tell from a lot of my comments, your understanding of the code relies heavily on your understanding of the methodology we've studied.

Thank you for working so hard on this, and feel free to reach out with ANY QUESTIONS no matter how simple. Even if you want us to explain what print(raw) means, we will. Office hours are hosted weekly with the goal of furthering your understanding and answering any questions, so feel free to stop by. Next week, we'll be working on classification charting, which may be a little more complicated, so I encourage you to refresh your memory on what SVM and KNN analysis is.

Have an amazing week/weekend, and thank you for completing this assignment!